In [12]:
import pandas as pd
#import matplotlib.pyplot as plt
data=pd.read_csv('pacifier.tsv',sep='\t',header=0)
date=pd.read_csv('自然日序列.csv',header=0)
data.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,40626522,R1A3ZUBR8TSAKY,B00793CZAE,572944212,"mary meyer wubbanub plush pacifier, lamb",Baby,5,0,0,N,Y,Love this!,Perfect match for the Gund Huggybuddy I bought...,8/31/2015
1,US,16290022,RLJNYBK4FGBYX,B003PCYMP4,911821018,wubbanub lamb infant pacifier,Baby,5,0,0,N,Y,Love 💕,My little girl love this paci contraption!,8/31/2015
2,US,10216509,R26QCW75C4JDOK,B003CK3LDI,392768822,wubbanub infant pacifier - giraffe,Baby,5,0,0,N,Y,Five Stars,My son loves this one and will only sleep if h...,8/31/2015
3,US,114040,R2E7N0TVLUHUDR,B003CK3LDI,392768822,wubbanub infant pacifier - giraffe,Baby,5,0,0,N,Y,Five Stars,Perfect,8/31/2015
4,US,27971579,R1SO9VMCIGZX3U,B003PCYMP4,911821018,wubbanub lamb infant pacifier,Baby,5,0,0,N,Y,Five Stars,Amazing addition to the nursery!,8/31/2015


In [13]:
#处理日期格式
def get_date(x):
    month,day,year=x.split('/')
    
    if int(day)<10:
        day='0'+day
    if int(month)<10:
        month='0'+month
        
    return year+month+day
data['review_date']=list(map(get_date,data['review_date']))

In [14]:
#得到data区间内的自然日序列
date['date']=list(map(str,date['date']))
data_star=data['star_rating'].groupby(data['star_rating']).count()
date=date[~(date['date']<min(list(data['review_date'])))]
date=date[~(date['date']>max(list(data['review_date'])))]
date.columns=['review_date']

In [15]:
#处理评论客户是否购买商品
data_verified_purchase_Y=data[data['verified_purchase']=='Y']#已购买商品的客户的评分
data_verified_purchase_N=data[~(data['verified_purchase']=='Y')]#未购买商品的客户的评分
#已购买商品的客户的评分分布
data_verified_purchase_Y_star=data_verified_purchase_Y['star_rating'].groupby(data_verified_purchase_Y['star_rating']).count()
print(data_verified_purchase_Y_star)
#已购买商品的客户的评分分布
data_verified_purchase_N_star=data_verified_purchase_N['star_rating'].groupby(data_verified_purchase_N['star_rating']).count()
data_verified_purchase_N_star.head()

star_rating
1     468
2     440
3     678
4    1255
5    6950
Name: star_rating, dtype: int64


star_rating
1     724
2     505
3     748
4    1461
5    5710
Name: star_rating, dtype: int64

In [16]:
star=data['star_rating'].groupby(data['review_date']).sum()#每日得到的评分合计
volume=data['star_rating'].groupby(data['review_date']).count()#每日评论数目，销量的代理变量，不含未购买商品的评论


star_=star.cumsum()#累计评分合计
volume_=volume.cumsum()#累计评论数

Favorable_rate=star_/(volume_*5)
volume__=pd.DataFrame([volume]).T
volume__=volume__.reset_index()

Favorable_rate=pd.DataFrame([Favorable_rate]).T
Favorable_rate=Favorable_rate.reset_index()

result1=pd.merge(Favorable_rate,volume__,on=['review_date'],how='left')
result1=result1.fillna(0)
result1.columns=['review_date', 'star_rating' , 'Cumulative sales volume']
result1['Cumulative sales volume']=result1['Cumulative sales volume'].cumsum()#
result1=pd.merge(date,result1,on='review_date',how='left')
result1=result1.fillna(method='ffill')
result1.tail()
result1.to_excel('./pacifier_result1.xlsx')

In [17]:
#得到每一类评分的累计数量
data1=data[['star_rating','review_date']]
data1['count']=1
datanew=data1.groupby(['star_rating','review_date']).sum()
datanew=datanew.reset_index()
datanew=datanew.set_index(['review_date','star_rating'])
datanew=datanew.unstack()
datanew=datanew.fillna(0)
datanew.columns=['1','2','3','4','5']
result2=datanew.cumsum()#每一类评分累计
result2=pd.merge(date,result2,on='review_date',how='left')
result2=result2.fillna(method='ffill')

<ipython-input-17-fa969f755305>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['count']=1


In [18]:
#得到已购买商品的评论的每一类评分的累计数量
data1=data_verified_purchase_Y[['star_rating','review_date']]
data1['count']=1
datanew=data1.groupby(['star_rating','review_date']).sum()
datanew=datanew.reset_index()
datanew=datanew.set_index(['review_date','star_rating'])
datanew=datanew.unstack()
datanew=datanew.fillna(0)
datanew.columns=['1','2','3','4','5']
result3=datanew.cumsum()#每一类评分累计
result3=pd.merge(date,result3,on='review_date',how='left')
result3=result3.fillna(method='ffill')

<ipython-input-18-f3b382bbc70f>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['count']=1


In [19]:
#得到未购买商品的评论的每一类评分的累计数量
data1=data_verified_purchase_N[['star_rating','review_date']]
data1['count']=1
datanew=data1.groupby(['star_rating','review_date']).sum()
datanew=datanew.reset_index()
datanew=datanew.set_index(['review_date','star_rating'])
datanew=datanew.unstack()
datanew=datanew.fillna(0)
datanew.columns=['1','2','3','4','5']
result4=datanew.cumsum()#每一类评分累计
result4=pd.merge(date,result4,on='review_date',how='left')
result4=result4.fillna(method='ffill')
result4=result4.fillna(0)
total_result=pd.concat([result1,result2,result3,result4],axis=1)
total_result.head()
total_result.to_csv('paifier_total_result.csv')

<ipython-input-19-fba219d7ff04>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['count']=1


In [10]:
#将文本转为情绪，范围-1到1
#在终端中pip install textblob -i https://pypi.tuna.tsinghua.edu.cn/simple



from textblob import TextBlob
def review_to_score(text):

    #text是英文文本，例如"I am happy today. I feel sad today."
    if type(text)==str:

        blob = TextBlob(text)

        score=blob.sentiment.polarity*2.5+2.5#情感分映射到0-5

    else:

        score=2.5#中性

    return score
data['review_headline_score']=list(map(review_to_score,data['review_headline']))
data['review_body_score']=list(map(review_to_score,data['review_body']))
result5=data[['review_headline','review_headline_score', 'review_body', 'review_body_score','review_date']]#用作评论和评论分数对比

In [11]:
result6=pd.concat([result1,result5['review_headline_score'],result5['review_body_score']],axis=1)
#total_result=pd.concat([result1,result2,result3,result4],axis=1)
#result6=result6['review_date', 'star_rating' , 'Cumulative sales volume','review_headline_score','review_body_score']
#result6=result6.fillna(method='ffill')
result6.to_csv('./pacifier_result6.csv')
result6.head()

,review_date,star_rating,Cumulative sales volume,review_headline_score,review_body_score
0,20030427,0.4,1.0,4.0625,5.000000
1,20030428,0.4,1.0,3.7500,3.046875
2,20030429,0.4,1.0,2.5000,2.500000
3,20030430,0.4,1.0,2.5000,5.000000
4,20030501,0.4,1.0,2.5000,4.375000
